In [15]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import tensorflow as tf

import multiprocessing as mp
# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

import pyaudio
import wave
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
learning_rate = 0.01
seq_length = 9 #layer
# train Parameters
X_dim = 442 #n_dim #X_hot.shape[2]

#seq_length = 7 #X_hot.shape[1]
output_dim = 1 #n_classes #Y_hot.shape[1]

hidden_dim = 2
learning_rate = 0.05

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, X_dim], name = 'X')
Y = tf.placeholder(tf.float32, [None, output_dim], name = 'Y')


# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Y_pred, labels = Y))

# optimizer
lr = tf.placeholder(tf.float32,shape=(), name='learning_rate')
train = tf.train.AdamOptimizer(lr).minimize(loss) #AdamOptimizer

# RMSE
targets = tf.placeholder(tf.float32, [None, output_dim] , name = 'targets')
predictions = tf.placeholder(tf.float32, [None, output_dim] , name = 'predictions')
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))


In [58]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
def showFreqTime(combine):
    for sample, filename, _ in combine:
        freqs, times, spectrogram = log_specgram(sample, SR)
        fig = plt.figure(figsize=(14, 10))
        ax1 = fig.add_subplot(211)
        ax1.set_title('Raw wave of ' + filename)
        ax1.set_ylabel('Amplitude')
        ax1.plot(np.linspace(0, len(sample)/SR, len(sample)), sample)

        ax2 = fig.add_subplot(212)
        ax2.imshow(spectrogram.T, aspect='auto', origin='lower', 
               extent=[times.min(), times.max(), freqs.min(), freqs.max()])
        ax2.set_yticks(freqs[::16])
        ax2.set_xticks(times[::16])
        ax2.set_title('Spectrogram of ' + filename)
        ax2.set_ylabel('Freqs in Hz')
        ax2.set_xlabel('Seconds')
def getStream():
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT, channels=1, rate=SAMPLE_RATE,
            input=True, frames_per_buffer=CHUNK_SIZE)
    wf = wave.open(WAVE_FILENAME, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(AUDIO_FORMAT))
    wf.setframerate(SR)
    
    t1 = time.time()
    # grab audio and timestamp
    #data = 
    #frames.append(data)
    audio = np.fromstring(stream.read(CHUNK_SIZE), np.int16)
    # write to the audio file
    wf.writeframes(b''.join(audio))
    audio = audio.astype(float)
    t2 = time.time()
    print("time: %.4f \t"%(t2-t1),end='')
    
    
    stream.stop_stream()
    stream.close()

    p.terminate()
    return audio
def printDescription(y_true, y_pred):
    p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
    print("F-Score:", round(f,3))
    print("Accuracy: ", accuracy_score(y_true, y_pred))

    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
def makeHot(dataX,seq_length):
    X_hot_list= []
    for i in range(0, dataX.shape[0] - seq_length+1):
        _x = dataX[i:i + seq_length]
        X_hot_list.append(_x)
    X_hot = np.array(X_hot_list[:])
    return X_hot

In [61]:
model_path_f = '../models/RNN/'
filename = 'my_RNN_model_S9_40'

#init 
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, model_path_f+filename)

mean = np.load('../data/Xy/RNN_mean.npy')
std = np.load('../data/Xy/RNN_std.npy')


import time
CHUNK_SIZE = 8192*4
AUDIO_FORMAT = pyaudio.paInt16
SR = 44100
SAMPLE_RATE = SR
WAVE_FILENAME = '../data/output.wav'
####
justone = True
while(justone):
    justone = True
    audio = getStream()
    
    #get Data
    freqs, times, spectrogram = log_specgram(audio, sample_rate)  
    #showFreqTime([[sample, filename1, SR]])  

    #Nomalization
    spectrogram = (spectrogram - mean) / std
    
    #stacked data
    X_hot = makeHot(spectrogram,seq_length)
    
    #recognization
    y_pred = sess.run(Y_pred,feed_dict={X: X_hot})
    #print(y_pred[:])
    
    y_pred[y_pred<0.5] = 0
    y_pred[y_pred>=0.5] = 1
    
    #printDescription(np.zeros(shape=[y_pred.shape[0]]), y_pred)
    
    #result
    result = y_pred[-1]
    if result == 1:
        print('The sound is Drone')
    else :
        print('THe sound isn\'t Drone')
sess.close()

INFO:tensorflow:Restoring parameters from ../models/RNN/my_RNN_model_S9_40


c:\users\jh\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
c:\users\jh\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


time: 0.7671 	THe sound isn't Drone
time: 0.7631 	THe sound isn't Drone
time: 0.7658 	THe sound isn't Drone
time: 0.7538 	THe sound isn't Drone
time: 0.7633 	THe sound isn't Drone
time: 0.7628 	THe sound isn't Drone
time: 0.7644 	THe sound isn't Drone
time: 0.7630 	THe sound isn't Drone
time: 0.7628 	THe sound isn't Drone
time: 0.7634 	THe sound isn't Drone
time: 0.7622 	THe sound isn't Drone
time: 0.7631 	THe sound isn't Drone
time: 0.7628 	THe sound isn't Drone
time: 0.7641 	THe sound isn't Drone
time: 0.7662 	THe sound isn't Drone
time: 0.7627 	THe sound isn't Drone
time: 0.7633 	THe sound isn't Drone
time: 0.7557 	THe sound isn't Drone
time: 0.7630 	THe sound isn't Drone
time: 0.7586 	THe sound isn't Drone
time: 0.7647 	THe sound isn't Drone
time: 0.7638 	THe sound isn't Drone
time: 0.7643 	THe sound isn't Drone
time: 0.7620 	THe sound isn't Drone
time: 0.7652 	THe sound isn't Drone
time: 0.7635 	THe sound isn't Drone
time: 0.7632 	THe sound isn't Drone
time: 0.7538 	THe sound isn'

KeyboardInterrupt: 